<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Instalar librerías necesarias
!pip install gradio openpyxl scikit-learn --quiet

import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import glob
import numpy as np
from io import BytesIO
import tempfile
import openpyxl
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Leer archivo CSV
csv_path = glob.glob("*.csv")
if not csv_path:
    raise FileNotFoundError("❌ No se encontró ningún archivo CSV en el entorno.")
csv_file = csv_path[0]
print(f"✅ Usando archivo: {csv_file}")

df = pd.read_csv(csv_file, skiprows=4)
columnas = ['Country Name'] + [str(year) for year in range(2000, 2023)]
df = df[columnas].dropna()
paises = sorted(df['Country Name'].unique().tolist())
matriz_pib = df.set_index('Country Name').apply(pd.to_numeric, errors='coerce')

# IA: Clustering
scaler = StandardScaler()
matriz_scaled = scaler.fit_transform(matriz_pib.fillna(0))
kmeans = KMeans(n_clusters=8, random_state=0).fit(matriz_scaled)
cluster_labels = pd.Series(kmeans.labels_, index=matriz_pib.index)

def sugerir(pais_ref):
    if pais_ref not in cluster_labels:
        return []
    cluster = cluster_labels[pais_ref]
    similares = cluster_labels[cluster_labels == cluster].index.tolist()
    similares.remove(pais_ref)
    return similares[:3]

# Bandera con nombre
def get_country_code(pais):
    import pycountry
    try:
        country = pycountry.countries.lookup(pais)
        return country.alpha_2.lower()
    except:
        return 'un'  # bandera genérica

def mostrar_bandera(pais):
    codigo = get_country_code(pais)
    return f"<img src='https://flagcdn.com/w40/{codigo}.png' style='vertical-align:middle; height:24px;'> <b>{pais}</b>"

# Comparación
def comparar_y_exportar(pais1, pais2):
    try:
        if pais1 == pais2:
            return None, None, "⚠️ Selecciona dos países diferentes.", None, ""

        datos1 = matriz_pib.loc[pais1].dropna()
        datos2 = matriz_pib.loc[pais2].dropna()

        años_comunes = datos1.index.intersection(datos2.index)
        datos1 = datos1.loc[años_comunes]
        datos2 = datos2.loc[años_comunes]

        comparativa = pd.DataFrame({
            'Año': años_comunes.astype(int),
            pais1: datos1.values,
            pais2: datos2.values
        })

        comparativa = comparativa.replace([np.inf, -np.inf], 0).fillna(0)
        comparativa[f"{pais1} Crec %"] = comparativa[pais1].pct_change()
        comparativa[f"{pais2} Crec %"] = comparativa[pais2].pct_change()
        comparativa = comparativa.round(2)

        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(comparativa['Año'], comparativa[pais1] / 1e9, label=pais1, marker='o')
        ax.plot(comparativa['Año'], comparativa[pais2] / 1e9, label=pais2, marker='o')
        ax.set_title(f'PIB: {pais1} vs {pais2}')
        ax.set_xlabel('Año')
        ax.set_ylabel('PIB (miles de millones USD)')
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:,.0f}'))
        ax.grid(True)
        ax.legend()
        plt.tight_layout()

        imagen = BytesIO()
        fig.savefig(imagen, format='png')
        imagen.seek(0)

        tmp_excel = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx").name
        tmp_img = BytesIO()
        fig.savefig(tmp_img, format='png')
        tmp_img.seek(0)

        with pd.ExcelWriter(tmp_excel, engine='openpyxl') as writer:
            comparativa.to_excel(writer, index=False, sheet_name='Comparación PIB')
            workbook = writer.book
            worksheet = workbook['Comparación PIB']

            for col_num, value in enumerate(comparativa.columns.values):
                worksheet.cell(row=1, column=col_num + 1, value=value)
                worksheet.column_dimensions[openpyxl.utils.get_column_letter(col_num + 1)].width = 18

            for row_num, row in comparativa.iterrows():
                for col_num, value in enumerate(row):
                    cell = worksheet.cell(row=row_num + 2, column=col_num + 1, value=value)
                    if "Crec" in comparativa.columns[col_num]:
                        cell.number_format = '0.00%'

            worksheet.add_image(openpyxl.drawing.image.Image(tmp_img), 'H2')

        sugeridos1 = sugerir(pais1)
        sugeridos2 = sugerir(pais2)

        texto = f"""🤖 IA Sugerencias:
- Países similares a **{pais1}**: {', '.join(sugeridos1)}
- Países similares a **{pais2}**: {', '.join(sugeridos2)}"""

        return fig, comparativa, texto, tmp_excel, ""
    except Exception as e:
        return None, None, f"❌ Error: {str(e)}", None, "⚠️ Ocurrió un error al comparar."

# Interfaz Gradio
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.HTML("""
        <style>
            body {
                background-image: url('https://upload.wikimedia.org/wikipedia/commons/8/80/World_map_-_low_resolution_gray_blue.png');
                background-size: cover;
                background-repeat: no-repeat;
                background-attachment: fixed;
            }
            .gradio-container {
                background-color: rgba(255, 255, 255, 0.93);
                border-radius: 16px;
                padding: 20px;
                margin: 20px auto;
                box-shadow: 0 8px 30px rgba(0,0,0,0.2);
            }
        </style>
    """)

    gr.Markdown("# 🌍 Comparador de PIB Mundial\nElige dos países para comparar su evolución económica:")

    with gr.Row():
        with gr.Column(scale=1):
            pais1_html = gr.HTML(value=mostrar_bandera(paises[0]))
            pais2_html = gr.HTML(value=mostrar_bandera(paises[1]))
            dropdown1 = gr.Dropdown(choices=paises, label="📌 País 1", value=paises[0])
            dropdown2 = gr.Dropdown(choices=paises, label="📌 País 2", value=paises[1])
            boton = gr.Button("📊 Comparar PIB", variant='primary')
            estado = gr.Markdown("")

            def actualizar_bandera(pais):
                return mostrar_bandera(pais)

            dropdown1.change(fn=actualizar_bandera, inputs=dropdown1, outputs=pais1_html)
            dropdown2.change(fn=actualizar_bandera, inputs=dropdown2, outputs=pais2_html)

        with gr.Column(scale=2):
            grafico = gr.Plot(label="📈 Gráfico")
            tabla = gr.Dataframe(label="📋 Tabla", interactive=False)

    with gr.Row():
        texto = gr.Markdown("🤖 Sugerencias IA")
        archivo = gr.File(label="📥 Descargar Excel generado", file_types=[".xlsx"])

    boton.click(fn=comparar_y_exportar, inputs=[dropdown1, dropdown2], outputs=[grafico, tabla, texto, archivo, estado])

demo.launch(share=True)




✅ Usando archivo: API_NY.GDP.MKTP.CD_DS2_es_csv_v2_1992.csv
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed1e40c59f84cff929.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')